In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd drive/MyDrive    

/content/drive/MyDrive


In [3]:
! cp -ri "/content/drive/MyDrive/training_data.zip" /content/

In [4]:
! unzip training_data.zip                       #unzipping training data

Archive:  training_data.zip
replace output_dir/27/1484265527.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
import torch
import torch.nn as nn
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pickle
import json
import os
import pandas as pd
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()
nltk.download('wordnet')

In [6]:
all_texts = {}

In [ ]:
# don't convert to lowercase as using bert base cased (case is important)
def preprocess(text):
    res = re.sub(r'[^\w\s]', '',text) # Remove punctuations
    res = " ".join(res.split())       # Remove whitespaces
    tokens = word_tokenize(res) 
    new_tokens = [lemmatiser.lemmatize(i) for i in tokens if not i in stop_words] # Remove stopwords and lemmatise
    res = ' '.join(new_tokens)
    return res

In [7]:
for folder in os.listdir('output_dir'):                        #extracting all text data
    for file in (os.listdir('output_dir/'+str(folder))):
        if file.endswith('.json'):
            f = json.load(open('output_dir/'+str(folder)+'/'+str(file)))
            name = file[:-5]
            all_texts[name] = f['text']
for i in all_texts:
    all_texts[i] = preprocess(all_texts[i])

In [8]:
!pip install -q git+https://github.com/huggingface/transformers.git             #dependencies
!pip install -U sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 42.0 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=6eb2bf267bddca44935e9d44ccbd074f8985f1a1a621e9a762d503074009df71
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [9]:
import pandas as pd                                                               
def func(name):                                                                 # train_examples data parsing 
  df = pd.read_csv(name)
  df.head()
  l1 = []
  l2 = []
  l3 = []
  for index,row in df.iterrows():
      id1,id2 = row['pair_id'].split('_')
      if id1 in all_texts and id2 in all_texts:
         l1.append(all_texts[id1])
         l2.append(all_texts[id2])
         l3.append((row['Overall']-1)/3)
  di = {'Text 1':l1, 'Text 2':l2, 'GT': l3}
  df = pd.DataFrame(di)
  train_examples=[]
  for i in range(len(df)):
    train_examples.append(InputExample(texts=[df["Text 1"][i], df["Text 2"][i]], label=float(df["GT"][i])))
  return(train_examples)

In [10]:
from sentence_transformers import SentenceTransformer, InputExample, losses     #importing different modules
from torch.utils.data import DataLoader

# For training



In [11]:

from sentence_transformers.cross_encoder import CrossEncoder                    #Bert training loading model
model1 = CrossEncoder('bert-base-multilingual-cased',num_labels=1)


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [12]:
import torch
import torch.nn as nn

In [13]:
train_examples=func("/content/drive/MyDrive/data.csv")
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)  # Keep it 8 for mbert and 1 for xlm
model1.fit(train_dataloader=train_dataloader,epochs=1,loss_fct = nn.MSELoss(),scheduler = "constantlr") # Vary epochs

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/618 [00:00<?, ?it/s]

In [15]:
import pickle                                                           #Saving trained model
filename = 'mbert_model.sav'
pickle.dump(model1, open(filename, 'wb'))